In [1]:
import sqlite3
import tkinter as tk
from tkinter import messagebox
import logging
import sys
import os
from datetime import datetime

# ---------------- LOGGING SETUP ----------------
logging.basicConfig(
    filename="soulsense.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Application started")

# ---------------- DATABASE INIT & MIGRATIONS ----------------
def ensure_scores_schema(cursor):
    cursor.execute("PRAGMA table_info(scores)")
    columns = [col[1] for col in cursor.fetchall()]
    # If table exists but missing age column, add it.
    if columns and "age" not in columns:
        logging.info("Migrating scores table: adding age column")
        cursor.execute("ALTER TABLE scores ADD COLUMN age INTEGER")
    # If table doesn't exist (columns empty), create will be handled below.

def ensure_question_bank_schema(cursor):
    logging.info("Ensuring question bank schema")

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS question_category (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT UNIQUE NOT NULL,
        description TEXT
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS question_bank (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        question_text TEXT NOT NULL,
        category_id INTEGER,
        difficulty INTEGER CHECK(difficulty BETWEEN 1 AND 5),
        weight REAL DEFAULT 1.0,
        min_age INTEGER,
        max_age INTEGER,
        is_active INTEGER DEFAULT 1,
        created_at TEXT,
        FOREIGN KEY (category_id) REFERENCES question_category(id)
    )
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS question_metadata (
        question_id INTEGER PRIMARY KEY,
        source TEXT,
        version TEXT,
        tags TEXT,
        FOREIGN KEY (question_id) REFERENCES question_bank(id)
    )
    """)
    
    
def ensure_responses_schema(cursor):
    """Ensure responses table exists and has required columns.
       Adds missing columns if table exists (safe for SQLite).
    """
    cursor.execute("PRAGMA table_info(responses)")
    cols = [col[1] for col in cursor.fetchall()]

    if not cols:
        # Table doesn't exist — create it
        logging.info("Creating responses table")
        cursor.execute("""
        CREATE TABLE responses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT,
            question_id INTEGER,
            response_value INTEGER,
            age_group TEXT,
            timestamp TEXT
        )
        """)
    else:
        # Table exists — ensure required columns are present, add if missing
        required = {
            "username": "TEXT",
            "question_id": "INTEGER",
            "response_value": "INTEGER",
            "age_group": "TEXT",
            "timestamp": "TEXT"
        }
        for col, coltype in required.items():
            if col not in cols:
                logging.info("Altering responses table: adding column %s", col)
                cursor.execute(f"ALTER TABLE responses ADD COLUMN {col} {coltype}")

try:
    conn = sqlite3.connect("soulsense_db")
    cursor = conn.cursor()

    # Create scores table if missing (backwards-compatible)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS scores (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT,
            total_score INTEGER
        )
    """)

    # Ensure scores schema has 'age' column (migration)
    ensure_scores_schema(cursor)

    # Ensure responses table and schema
    ensure_responses_schema(cursor)
    
    # Ensure question bank schema (Issue #7)
    ensure_question_bank_schema(cursor)

    conn.commit()
    logging.info("Database initialized successfully")

except Exception:
    logging.critical("Fatal error during database initialization", exc_info=True)
    try:
        messagebox.showerror("Fatal Error", "Unable to initialize database.")
    except Exception:
        pass
    sys.exit(1)

# ---------------- LOAD QUESTIONS ----------------
try:
    BASE_DIR = os.getcwd()
    QUESTION_FILE = os.path.join(BASE_DIR, "question_bank.txt")

    logging.info("Loading questions from %s", QUESTION_FILE)

    with open(QUESTION_FILE, "r", encoding="utf-8") as f:
        questions = [line.strip() for line in f if line.strip()]

    # optionally limit for quick tests:
    questions = questions[:10]

    if not questions:
        raise ValueError("Question bank is empty")

    logging.info("Questions loaded successfully | count=%s", len(questions))

except Exception:
    logging.critical("Failed to load questions", exc_info=True)
    try:
        messagebox.showerror(
            "Fatal Error",
            "Question bank could not be loaded.\nApplication will close."
        )
    except Exception:
        pass
    sys.exit(1)

# ---------------- HELPERS ----------------
def compute_age_group(age):
    """Return age group string for storage (simple buckets)."""
    if age is None:
        return "unknown"
    try:
        a = int(age)
    except Exception:
        return "unknown"
    if a < 18:
        return "child"
    if a < 65:
        return "adult"
    return "senior"

# ---------------- GUI APPLICATION ----------------
class SoulSenseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Soul Sense EQ Test")

        self.username = ""
        self.age = None
        self.current_question = 0
        self.responses = []  # in-memory list of numeric answers (for final score)
        self.age_group = None

        self.create_username_screen()

    # ---------- SCREENS ----------
    def create_username_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Enter Your Name:", font=("Arial", 14)).pack(pady=10)
        self.name_entry = tk.Entry(self.root, font=("Arial", 14))
        self.name_entry.pack(pady=5)

        tk.Label(self.root, text="Enter Your Age (optional):", font=("Arial", 14)).pack(pady=5)
        self.age_entry = tk.Entry(self.root, font=("Arial", 14))
        self.age_entry.pack(pady=5)

        tk.Button(self.root, text="Start Test", command=self.start_test).pack(pady=15)

    # ---------- VALIDATION ----------
    def validate_name_input(self, name):
        if not name:
            return False, "Please enter your name."
        if not all(c.isalpha() or c.isspace() for c in name):
            return False, "Name must contain only letters and spaces."
        return True, None

    def validate_age_input(self, age_str):
        if age_str == "":
            return True, None, None
        try:
            age = int(age_str)
            if not (1 <= age <= 120):
                return False, None, "Age must be between 1 and 120."
            return True, age, None
        except ValueError:
            return False, None, "Age must be numeric."

    # ---------- FLOW ----------
    def start_test(self):
        try:
            self.username = self.name_entry.get().strip()
            age_str = self.age_entry.get().strip()

            ok, err = self.validate_name_input(self.username)
            if not ok:
                logging.warning("Invalid name input: %s", self.username)
                messagebox.showwarning("Input Error", err)
                return

            ok, age, err = self.validate_age_input(age_str)
            if not ok:
                logging.warning("Invalid age input: %s", age_str)
                messagebox.showwarning("Input Error", err)
                return

            self.age = age
            self.age_group = compute_age_group(age)
            logging.info("User session started | username=%s | age=%s | age_group=%s", self.username, self.age, self.age_group)

            self.show_question()

        except Exception:
            self.handle_fatal_error("Error starting test")

    def show_question(self):
        try:
            self.clear_screen()

            if self.current_question >= len(questions):
                self.finish_test()
                return

            q = questions[self.current_question]
            tk.Label(self.root, text=f"Q{self.current_question+1}: {q}", wraplength=400).pack(pady=20)

            self.answer_var = tk.IntVar()
            for val, txt in enumerate(["Never", "Sometimes", "Often", "Always"], 1):
                tk.Radiobutton(self.root, text=f"{txt} ({val})", variable=self.answer_var, value=val).pack(anchor="w", padx=50)

            tk.Button(self.root, text="Next", command=self.save_answer).pack(pady=15)

        except Exception:
            self.handle_fatal_error("Error displaying question")

    def save_answer(self):
        try:
            ans = self.answer_var.get()
            if ans == 0:
                logging.warning("No answer selected | user=%s | q=%s", self.username, self.current_question + 1)
                messagebox.showwarning("Input Error", "Please select an answer.")
                return

            # store in-memory for final calculation
            self.responses.append(ans)

            # persist per-question record to DB
            qid = self.current_question + 1
            ts = datetime.utcnow().isoformat()
            try:
                cursor.execute(
                    "INSERT INTO responses (username, question_id, response_value, age_group, timestamp) VALUES (?, ?, ?, ?, ?)",
                    (self.username, qid, ans, self.age_group, ts)
                )
                conn.commit()
                logging.info("Response stored | user=%s | qid=%s | value=%s", self.username, qid, ans)
            except Exception:
                logging.error("Failed to store question-wise response", exc_info=True)
                messagebox.showerror("Database Error", "Failed to save your response. Continuing in-memory.")
                # continue — keep in-memory value so user can finish attempt

            self.current_question += 1
            self.show_question()

        except Exception:
            self.handle_fatal_error("Error saving answer")

    def finish_test(self):
        try:
            total_score = sum(self.responses)

            # Save final score (age column is present due to migration)
            try:
                cursor.execute(
                    "INSERT INTO scores (username, age, total_score) VALUES (?, ?, ?)",
                    (self.username, self.age, total_score)
                )
                conn.commit()
                logging.info("Final score saved | user=%s | score=%s", self.username, total_score)
            except Exception:
                logging.error("Failed to store final score", exc_info=True)
                messagebox.showerror("Database Error", "Failed to save final score. It may not be recorded.")

            # interpretation (same thresholds)
            interpretation = (
                "Excellent Emotional Intelligence!" if total_score >= 65 else
                "Good Emotional Intelligence." if total_score >= 50 else
                "Average Emotional Intelligence." if total_score >= 35 else
                "You may want to work on your Emotional Intelligence."
            )

            self.clear_screen()
            tk.Label(self.root, text=f"Thank you, {self.username}!", font=("Arial", 16)).pack(pady=10)
            tk.Label(self.root, text=f"Your total EQ score is: {total_score} / {len(self.responses)*4}", font=("Arial", 14)).pack(pady=10)
            tk.Label(self.root, text=interpretation, font=("Arial", 14), fg="blue").pack(pady=10)

            # Show all results (preserve original behavior)
            self.show_all_results()

            tk.Button(self.root, text="Exit", command=self.force_exit, font=("Arial", 12)).pack(pady=20)

        except Exception:
            self.handle_fatal_error("Failed to finish test")

    # ---------- SHOW / EXPORT RESULTS ----------
    def show_all_results(self):
        # Display final scores (as original) and recent response records
        try:
            print("\n--- All EQ Test Results ---")
            print(f"{'Username':<20} {'Age':<10} {'Total Score'}")
            print("-" * 45)

            cursor.execute("SELECT username, age, total_score FROM scores")
            rows = cursor.fetchall()
            for row in rows:
                age_display = str(row[1]) if row[1] is not None else "N/A"
                print(f"{row[0]:<20} {age_display:<10} {row[2]}")

            # Also show recent responses for the current user (if any)
            print("\n--- Recent Responses (last 100 rows) ---")
            cursor.execute("SELECT username, question_id, response_value, age_group, timestamp FROM responses ORDER BY id DESC LIMIT 100")
            resp_rows = cursor.fetchall()
#             for r in resp_rows:
#                 print(f"user={r[0]:<12} q={r[1]:<2} val={r[2]:<2} age_group={r[3]:<8} ts={r[4]}")
            for r in resp_rows:
                user = r[0] or "N/A"
                qid = r[1] if r[1] is not None else "-"
                val = r[2] if r[2] is not None else "-"
                age_grp = r[3] or "unknown"
                ts = r[4] or "N/A"

                print(
                    f"user={str(user):<12} "
                    f"q={str(qid):<2} "
                    f"val={str(val):<2} "
                    f"age_group={str(age_grp):<8} "
                    f"ts={ts}"
                )

        except Exception:
            logging.error("Failed to fetch results for display", exc_info=True)
            messagebox.showerror("Error", "Unable to fetch results for display.")

    # ---------- EXIT ----------
    def force_exit(self):
        try:
            conn.close()
            logging.info("Database connection closed")
        except Exception:
            logging.error("Error closing database", exc_info=True)
        finally:
            logging.info("Application exited")
            try:
                self.root.destroy()
            except Exception:
                pass
            # In Jupyter, don't sys.exit() aggressively; exit if running as script
            if not hasattr(sys, "ps1"):
                sys.exit(0)

    def handle_fatal_error(self, msg):
        logging.critical(msg, exc_info=True)
        try:
            messagebox.showerror("Fatal Error", "A critical error occurred. App will close.")
        except Exception:
            pass
        self.force_exit()

    def clear_screen(self):
        for w in self.root.winfo_children():
            w.destroy()

# ---------------- MAIN ----------------
try:
    root = tk.Tk()
    root.geometry("500x350")
    app = SoulSenseApp(root)
    root.protocol("WM_DELETE_WINDOW", app.force_exit)
    root.mainloop()

except Exception:
    logging.critical("Unhandled crash", exc_info=True)
    try:
        conn.close()
    except Exception:
        pass
    sys.exit(1)



--- All EQ Test Results ---
Username             Age        Total Score
---------------------------------------------
Test                 N/A        63
213124eqwwdxas       N/A        63
sad                  N/A        63
Aleena               N/A        78
Aleena               N/A        62
Aleena               23         36
Aleena               23         37
Aleena               23         32
Aleenatest           23         31
AleenaMajima         23         37
AleenaHaroldPeter    22         36
AleenaHaroldPeter    23         37
Aleena               21         34

--- Recent Responses (last 100 rows) ---
user=Aleena       q=10 val=3  age_group=adult    ts=2026-01-05T05:06:08.643581
user=Aleena       q=9  val=4  age_group=adult    ts=2026-01-05T05:06:07.253213
user=Aleena       q=8  val=3  age_group=adult    ts=2026-01-05T05:06:05.643682
user=Aleena       q=7  val=4  age_group=adult    ts=2026-01-05T05:06:04.200408
user=Aleena       q=6  val=3  age_group=adult    ts=2026-01-05T05:06

# 12

In [5]:
import sqlite3
import tkinter as tk
from tkinter import messagebox
import logging
import sys
import os
from datetime import datetime

# ---------------- LOGGING SETUP ----------------
logging.basicConfig(
    filename="soulsense.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Application started")

# ---------------- DATABASE INIT & MIGRATIONS ----------------
def ensure_scores_schema(cursor):
    cursor.execute("PRAGMA table_info(scores)")
    columns = [col[1] for col in cursor.fetchall()]
    # If table exists but missing age column, add it.
    if columns and "age" not in columns:
        logging.info("Migrating scores table: adding age column")
        cursor.execute("ALTER TABLE scores ADD COLUMN age INTEGER")
    # If table doesn't exist (columns empty), create will be handled below.

def ensure_responses_schema(cursor):
    """Ensure responses table exists and has required columns.
       Adds missing columns if table exists (safe for SQLite).
    """
    cursor.execute("PRAGMA table_info(responses)")
    cols = [col[1] for col in cursor.fetchall()]

    if not cols:
        # Table doesn't exist — create it
        logging.info("Creating responses table")
        cursor.execute("""
        CREATE TABLE responses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT,
            question_id INTEGER,
            response_value INTEGER,
            age_group TEXT,
            timestamp TEXT
        )
        """)
    else:
        # Table exists — ensure required columns are present, add if missing
        required = {
            "username": "TEXT",
            "question_id": "INTEGER",
            "response_value": "INTEGER",
            "age_group": "TEXT",
            "timestamp": "TEXT"
        }
        for col, coltype in required.items():
            if col not in cols:
                logging.info("Altering responses table: adding column %s", col)
                cursor.execute(f"ALTER TABLE responses ADD COLUMN {col} {coltype}")

try:
    conn = sqlite3.connect("soulsense_db")
    cursor = conn.cursor()

    # Create scores table if missing (backwards-compatible)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS scores (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT,
            total_score INTEGER
        )
    """)

    # Ensure scores schema has 'age' column (migration)
    ensure_scores_schema(cursor)

    # Ensure responses table and schema
    ensure_responses_schema(cursor)

    conn.commit()
    logging.info("Database initialized successfully")

except Exception:
    logging.critical("Fatal error during database initialization", exc_info=True)
    try:
        messagebox.showerror("Fatal Error", "Unable to initialize database.")
    except Exception:
        pass
    sys.exit(1)

# ---------------- LOAD QUESTIONS ----------------
try:
    BASE_DIR = os.getcwd()
    QUESTION_FILE = os.path.join(BASE_DIR, "question_bank.txt")

    logging.info("Loading questions from %s", QUESTION_FILE)

    with open(QUESTION_FILE, "r", encoding="utf-8") as f:
        questions = [line.strip() for line in f if line.strip()]

    # optionally limit for quick tests:
    questions = questions[:10]

    if not questions:
        raise ValueError("Question bank is empty")

    logging.info("Questions loaded successfully | count=%s", len(questions))

except Exception:
    logging.critical("Failed to load questions", exc_info=True)
    try:
        messagebox.showerror(
            "Fatal Error",
            "Question bank could not be loaded.\nApplication will close."
        )
    except Exception:
        pass
    sys.exit(1)

# ---------------- HELPERS ----------------
def compute_age_group(age):
    """Return age group string for storage (simple buckets)."""
    if age is None:
        return "unknown"
    try:
        a = int(age)
    except Exception:
        return "unknown"
    if a < 18:
        return "child"
    if a < 65:
        return "adult"
    return "senior"

# ---------------- GUI APPLICATION ----------------
class SoulSenseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Soul Sense EQ Test")

        self.username = ""
        self.age = None
        self.current_question = 0
        self.responses = []  # in-memory list of numeric answers (for final score)
        self.age_group = None

        self.create_username_screen()

    # ---------- SCREENS ----------
    def create_username_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Enter Your Name:", font=("Arial", 14)).pack(pady=10)
        self.name_entry = tk.Entry(self.root, font=("Arial", 14))
        self.name_entry.pack(pady=5)

        tk.Label(self.root, text="Enter Your Age (optional):", font=("Arial", 14)).pack(pady=5)
        self.age_entry = tk.Entry(self.root, font=("Arial", 14))
        self.age_entry.pack(pady=5)

        tk.Button(self.root, text="Start Test", command=self.start_test).pack(pady=15)

    # ---------- VALIDATION ----------
    def validate_name_input(self, name):
        if not name:
            return False, "Please enter your name."
        if not all(c.isalpha() or c.isspace() for c in name):
            return False, "Name must contain only letters and spaces."
        return True, None

    def validate_age_input(self, age_str):
        if age_str == "":
            return True, None, None
        try:
            age = int(age_str)
            if not (1 <= age <= 120):
                return False, None, "Age must be between 1 and 120."
            return True, age, None
        except ValueError:
            return False, None, "Age must be numeric."

    # ---------- FLOW ----------
    def start_test(self):
        try:
            self.username = self.name_entry.get().strip()
            age_str = self.age_entry.get().strip()

            ok, err = self.validate_name_input(self.username)
            if not ok:
                logging.warning("Invalid name input: %s", self.username)
                messagebox.showwarning("Input Error", err)
                return

            ok, age, err = self.validate_age_input(age_str)
            if not ok:
                logging.warning("Invalid age input: %s", age_str)
                messagebox.showwarning("Input Error", err)
                return

            self.age = age
            self.age_group = compute_age_group(age)
            logging.info("User session started | username=%s | age=%s | age_group=%s", self.username, self.age, self.age_group)

            self.show_question()

        except Exception:
            self.handle_fatal_error("Error starting test")

    def show_question(self):
        try:
            self.clear_screen()

            if self.current_question >= len(questions):
                self.finish_test()
                return

            q = questions[self.current_question]
            tk.Label(self.root, text=f"Q{self.current_question+1}: {q}", wraplength=400).pack(pady=20)

            self.answer_var = tk.IntVar()
            for val, txt in enumerate(["Never", "Sometimes", "Often", "Always"], 1):
                tk.Radiobutton(self.root, text=f"{txt} ({val})", variable=self.answer_var, value=val).pack(anchor="w", padx=50)

            tk.Button(self.root, text="Next", command=self.save_answer).pack(pady=15)

        except Exception:
            self.handle_fatal_error("Error displaying question")

    def save_answer(self):
        try:
            ans = self.answer_var.get()
            if ans == 0:
                logging.warning("No answer selected | user=%s | q=%s", self.username, self.current_question + 1)
                messagebox.showwarning("Input Error", "Please select an answer.")
                return

            # store in-memory for final calculation
            self.responses.append(ans)

            # persist per-question record to DB
            qid = self.current_question + 1
            ts = datetime.utcnow().isoformat()
            try:
                cursor.execute(
                    "INSERT INTO responses (username, question_id, response_value, age_group, timestamp) VALUES (?, ?, ?, ?, ?)",
                    (self.username, qid, ans, self.age_group, ts)
                )
                conn.commit()
                logging.info("Response stored | user=%s | qid=%s | value=%s", self.username, qid, ans)
            except Exception:
                logging.error("Failed to store question-wise response", exc_info=True)
                messagebox.showerror("Database Error", "Failed to save your response. Continuing in-memory.")
                # continue — keep in-memory value so user can finish attempt

            self.current_question += 1
            self.show_question()

        except Exception:
            self.handle_fatal_error("Error saving answer")

    def finish_test(self):
        try:
            total_score = sum(self.responses)

            # Save final score (age column is present due to migration)
            try:
                cursor.execute(
                    "INSERT INTO scores (username, age, total_score) VALUES (?, ?, ?)",
                    (self.username, self.age, total_score)
                )
                conn.commit()
                logging.info("Final score saved | user=%s | score=%s", self.username, total_score)
            except Exception:
                logging.error("Failed to store final score", exc_info=True)
                messagebox.showerror("Database Error", "Failed to save final score. It may not be recorded.")

            # interpretation (same thresholds)
            interpretation = (
                "Excellent Emotional Intelligence!" if total_score >= 65 else
                "Good Emotional Intelligence." if total_score >= 50 else
                "Average Emotional Intelligence." if total_score >= 35 else
                "You may want to work on your Emotional Intelligence."
            )

            self.clear_screen()
            tk.Label(self.root, text=f"Thank you, {self.username}!", font=("Arial", 16)).pack(pady=10)
            tk.Label(self.root, text=f"Your total EQ score is: {total_score} / {len(self.responses)*4}", font=("Arial", 14)).pack(pady=10)
            tk.Label(self.root, text=interpretation, font=("Arial", 14), fg="blue").pack(pady=10)

            # Show all results (preserve original behavior)
            self.show_all_results()

            tk.Button(self.root, text="Exit", command=self.force_exit, font=("Arial", 12)).pack(pady=20)

        except Exception:
            self.handle_fatal_error("Failed to finish test")

    # ---------- SHOW / EXPORT RESULTS ----------
    def show_all_results(self):
        # Display final scores (as original) and recent response records
        try:
            print("\n--- All EQ Test Results ---")
            print(f"{'Username':<20} {'Age':<10} {'Total Score'}")
            print("-" * 45)

            cursor.execute("SELECT username, age, total_score FROM scores")
            rows = cursor.fetchall()
            for row in rows:
                age_display = str(row[1]) if row[1] is not None else "N/A"
                print(f"{row[0]:<20} {age_display:<10} {row[2]}")

            # Also show recent responses for the current user (if any)
            print("\n--- Recent Responses (last 100 rows) ---")
            cursor.execute("SELECT username, question_id, response_value, age_group, timestamp FROM responses ORDER BY id DESC LIMIT 100")
            resp_rows = cursor.fetchall()
#             for r in resp_rows:
#                 print(f"user={r[0]:<12} q={r[1]:<2} val={r[2]:<2} age_group={r[3]:<8} ts={r[4]}")
            for r in resp_rows:
                user = r[0] or "N/A"
                qid = r[1] if r[1] is not None else "-"
                val = r[2] if r[2] is not None else "-"
                age_grp = r[3] or "unknown"
                ts = r[4] or "N/A"

                print(
                    f"user={str(user):<12} "
                    f"q={str(qid):<2} "
                    f"val={str(val):<2} "
                    f"age_group={str(age_grp):<8} "
                    f"ts={ts}"
                )

        except Exception:
            logging.error("Failed to fetch results for display", exc_info=True)
            messagebox.showerror("Error", "Unable to fetch results for display.")

    # ---------- EXIT ----------
    def force_exit(self):
        try:
            conn.close()
            logging.info("Database connection closed")
        except Exception:
            logging.error("Error closing database", exc_info=True)
        finally:
            logging.info("Application exited")
            try:
                self.root.destroy()
            except Exception:
                pass
            # In Jupyter, don't sys.exit() aggressively; exit if running as script
            if not hasattr(sys, "ps1"):
                sys.exit(0)

    def handle_fatal_error(self, msg):
        logging.critical(msg, exc_info=True)
        try:
            messagebox.showerror("Fatal Error", "A critical error occurred. App will close.")
        except Exception:
            pass
        self.force_exit()

    def clear_screen(self):
        for w in self.root.winfo_children():
            w.destroy()

# ---------------- MAIN ----------------
try:
    root = tk.Tk()
    root.geometry("500x350")
    app = SoulSenseApp(root)
    root.protocol("WM_DELETE_WINDOW", app.force_exit)
    root.mainloop()

except Exception:
    logging.critical("Unhandled crash", exc_info=True)
    try:
        conn.close()
    except Exception:
        pass
    sys.exit(1)



--- All EQ Test Results ---
Username             Age        Total Score
---------------------------------------------
Test                 N/A        63
213124eqwwdxas       N/A        63
sad                  N/A        63
Aleena               N/A        78
Aleena               N/A        62
Aleena               23         36
Aleena               23         37
Aleena               23         32
Aleenatest           23         31
AleenaMajima         23         37

--- Recent Responses (last 100 rows) ---
user=AleenaMajima q=10 val=4  age_group=adult    ts=2026-01-04T14:18:38.693886
user=AleenaMajima q=9  val=2  age_group=adult    ts=2026-01-04T14:18:37.252926
user=AleenaMajima q=8  val=4  age_group=adult    ts=2026-01-04T14:18:35.566058
user=AleenaMajima q=7  val=4  age_group=adult    ts=2026-01-04T14:18:33.135065
user=AleenaMajima q=6  val=4  age_group=adult    ts=2026-01-04T14:18:31.623708
user=AleenaMajima q=5  val=4  age_group=adult    ts=2026-01-04T14:18:29.877852
user=AleenaMaji

In [8]:
# To Check the value stored in the previous session
# import pandas as pd
# import sqlite3

# conn = sqlite3.connect("soulsense_db")

# df = pd.read_sql_query("SELECT * FROM responses", conn)
# df

# 
# Delete all records from the table
# conn.execute("DELETE FROM responses")
# conn.commit()


,id,username,question_id,response_value,age_group,timestamp


In [2]:
# To Check the value stored in the previous session
# import pandas as pd
# import sqlite3

# conn = sqlite3.connect("soulsense_db")

# df = pd.read_sql_query("SELECT * FROM responses", conn)
# df


# Delete all records from the table
# conn.execute("DELETE FROM responses")
# conn.commit()


,id,username,question_id,response_value,age_group,timestamp
0,11,AleenaHaroldPeter,1,4,adult,2026-01-04T14:46:09.648121
1,12,AleenaHaroldPeter,2,4,adult,2026-01-04T14:46:11.196941
2,13,AleenaHaroldPeter,3,4,adult,2026-01-04T14:46:12.860119
3,14,AleenaHaroldPeter,4,3,adult,2026-01-04T14:46:14.759821
4,15,AleenaHaroldPeter,5,4,adult,2026-01-04T14:46:16.703155
5,16,AleenaHaroldPeter,6,4,adult,2026-01-04T14:46:18.844247
6,17,AleenaHaroldPeter,7,2,adult,2026-01-04T14:46:20.801900
7,18,AleenaHaroldPeter,8,4,adult,2026-01-04T14:46:22.551165
8,19,AleenaHaroldPeter,9,3,adult,2026-01-04T14:46:24.023220
9,20,AleenaHaroldPeter,10,4,adult,2026-01-04T14:46:25.464883


In [3]:

df = pd.read_sql_query("SELECT * FROM question_bank", conn)
df

,id,question_text,category_id,difficulty,weight,min_age,max_age,is_active,created_at
